# ClickGraph Relationship Testing

This notebook tests the relationship functionality in ClickGraph using the social network view configuration.

## Configuration Overview

The `examples/social_network_view.yaml` defines:
- **Nodes**: user, post, customer, product
- **Relationships**: AUTHORED, LIKED, FOLLOWS, PURCHASED

Server startup shows:
```
- Loaded 4 relationship types: ["AUTHORED", "LIKED", "FOLLOWS", "PURCHASED"]
```

Our goal is to test if these relationship types can generate proper JOIN SQL queries.

## Test 1: AUTHORED Relationship (User → Post)

The AUTHORED relationship connects users to posts they have written.

In [5]:
import requests
import json

def test_cypher_query(query, description, sql_only=True):
    """Test a Cypher query and return the result"""
    url = "http://localhost:8081/query"  # Fixed endpoint
    headers = {"Content-Type": "application/json"}
    data = {"query": query, "sql_only": sql_only}  # SQL-only mode to avoid ClickHouse execution
    
    print(f"\n{description}")
    print(f"Query: {query}")
    print("-" * 80)
    
    try:
        response = requests.post(url, headers=headers, json=data)
        print(f"Status Code: {response.status_code}")
        
        if response.status_code == 200:
            result = response.json()
            print(f"Response: {json.dumps(result, indent=2)}")
            return True, result
        else:
            print(f"Error Response: {response.text}")
            return False, None
            
    except Exception as e:
        print(f"Connection Error: {e}")
        return False, None

# Test AUTHORED relationship
query = "MATCH (u:user)-[r:AUTHORED]->(p:post) RETURN u.name, p.title LIMIT 5"
test_cypher_query(query, "Testing AUTHORED relationship between users and posts")


Testing AUTHORED relationship between users and posts
Query: MATCH (u:user)-[r:AUTHORED]->(p:post) RETURN u.name, p.title LIMIT 5
--------------------------------------------------------------------------------
Status Code: 200
Response: {
  "cypher_query": "MATCH (u:user)-[r:AUTHORED]->(p:post) RETURN u.name, p.title LIMIT 5",
  "generated_sql": "WITH user_u AS (\n    SELECT \n      u.name, \n      u.user_id\nFROM user AS t\n), \nAUTHORED_r AS (\n    SELECT \n      u.from_user AS from_id, \n      p.to_post AS to_id\nFROM AUTHORED AS t\nWHERE t.to_id IN (SELECT u.user_id FROM user_u AS t)\n)\nSELECT \n      u.name, \n      p.title\nFROM post AS p\nINNER JOIN AUTHORED_r AS r ON r.from_id = p.post_id\nINNER JOIN user_u AS u ON u.user_id = r.to_id\nLIMIT  5",
  "execution_mode": "sql_only"
}
Status Code: 200
Response: {
  "cypher_query": "MATCH (u:user)-[r:AUTHORED]->(p:post) RETURN u.name, p.title LIMIT 5",
  "generated_sql": "WITH user_u AS (\n    SELECT \n      u.name, \n      u.user_

(True,
 {'cypher_query': 'MATCH (u:user)-[r:AUTHORED]->(p:post) RETURN u.name, p.title LIMIT 5',
  'generated_sql': 'WITH user_u AS (\n    SELECT \n      u.name, \n      u.user_id\nFROM user AS t\n), \nAUTHORED_r AS (\n    SELECT \n      u.from_user AS from_id, \n      p.to_post AS to_id\nFROM AUTHORED AS t\nWHERE t.to_id IN (SELECT u.user_id FROM user_u AS t)\n)\nSELECT \n      u.name, \n      p.title\nFROM post AS p\nINNER JOIN AUTHORED_r AS r ON r.from_id = p.post_id\nINNER JOIN user_u AS u ON u.user_id = r.to_id\nLIMIT  5',
  'execution_mode': 'sql_only'})

## Test 2: FOLLOWS Relationship (User → User)

The FOLLOWS relationship represents user-to-user following connections in a social network.

In [6]:
# Test FOLLOWS relationship
query = "MATCH (u1:user)-[r:FOLLOWS]->(u2:user) RETURN u1.name AS follower, u2.name AS following LIMIT 5"
test_cypher_query(query, "Testing FOLLOWS relationship between users")


Testing FOLLOWS relationship between users
Query: MATCH (u1:user)-[r:FOLLOWS]->(u2:user) RETURN u1.name AS follower, u2.name AS following LIMIT 5
--------------------------------------------------------------------------------
Status Code: 200
Response: {
  "cypher_query": "MATCH (u1:user)-[r:FOLLOWS]->(u2:user) RETURN u1.name AS follower, u2.name AS following LIMIT 5",
  "generated_sql": "WITH user_u1 AS (\n    SELECT \n      u.name, \n      u.user_id\nFROM user AS t\n), \nFOLLOWS_r AS (\n    SELECT \n      u.from_user AS from_id, \n      u.to_user AS to_id\nFROM FOLLOWS AS t\nWHERE t.to_id IN (SELECT u.user_id FROM user_u1 AS t)\n)\nSELECT \n      u1.name AS follower, \n      u2.name AS following\nFROM user AS u2\nINNER JOIN FOLLOWS_r AS r ON r.to_id = u2.user_id\nINNER JOIN user_u1 AS u1 ON u1.user_id = r.from_id\nLIMIT  5",
  "execution_mode": "sql_only"
}


(True,
 {'cypher_query': 'MATCH (u1:user)-[r:FOLLOWS]->(u2:user) RETURN u1.name AS follower, u2.name AS following LIMIT 5',
  'generated_sql': 'WITH user_u1 AS (\n    SELECT \n      u.name, \n      u.user_id\nFROM user AS t\n), \nFOLLOWS_r AS (\n    SELECT \n      u.from_user AS from_id, \n      u.to_user AS to_id\nFROM FOLLOWS AS t\nWHERE t.to_id IN (SELECT u.user_id FROM user_u1 AS t)\n)\nSELECT \n      u1.name AS follower, \n      u2.name AS following\nFROM user AS u2\nINNER JOIN FOLLOWS_r AS r ON r.to_id = u2.user_id\nINNER JOIN user_u1 AS u1 ON u1.user_id = r.from_id\nLIMIT  5',
  'execution_mode': 'sql_only'})

## Test 3: LIKED Relationship (User → Post)

The LIKED relationship represents users liking posts.

In [7]:
# Test LIKED relationship
query = "MATCH (u:user)-[r:LIKED]->(p:post) RETURN u.name AS liker, p.title AS liked_post LIMIT 5"
test_cypher_query(query, "Testing LIKED relationship between users and posts")


Testing LIKED relationship between users and posts
Query: MATCH (u:user)-[r:LIKED]->(p:post) RETURN u.name AS liker, p.title AS liked_post LIMIT 5
--------------------------------------------------------------------------------
Status Code: 200
Response: {
  "cypher_query": "MATCH (u:user)-[r:LIKED]->(p:post) RETURN u.name AS liker, p.title AS liked_post LIMIT 5",
  "generated_sql": "WITH user_u AS (\n    SELECT \n      u.name, \n      u.user_id\nFROM user AS t\n), \nLIKED_r AS (\n    SELECT \n      u.from_user AS from_id, \n      p.to_post AS to_id\nFROM LIKED AS t\nWHERE t.to_id IN (SELECT u.user_id FROM user_u AS t)\n)\nSELECT \n      u.name AS liker, \n      p.title AS liked_post\nFROM post AS p\nINNER JOIN LIKED_r AS r ON r.from_id = p.post_id\nINNER JOIN user_u AS u ON u.user_id = r.to_id\nLIMIT  5",
  "execution_mode": "sql_only"
}
Status Code: 200
Response: {
  "cypher_query": "MATCH (u:user)-[r:LIKED]->(p:post) RETURN u.name AS liker, p.title AS liked_post LIMIT 5",
  "genera

(True,
 {'cypher_query': 'MATCH (u:user)-[r:LIKED]->(p:post) RETURN u.name AS liker, p.title AS liked_post LIMIT 5',
  'generated_sql': 'WITH user_u AS (\n    SELECT \n      u.name, \n      u.user_id\nFROM user AS t\n), \nLIKED_r AS (\n    SELECT \n      u.from_user AS from_id, \n      p.to_post AS to_id\nFROM LIKED AS t\nWHERE t.to_id IN (SELECT u.user_id FROM user_u AS t)\n)\nSELECT \n      u.name AS liker, \n      p.title AS liked_post\nFROM post AS p\nINNER JOIN LIKED_r AS r ON r.from_id = p.post_id\nINNER JOIN user_u AS u ON u.user_id = r.to_id\nLIMIT  5',
  'execution_mode': 'sql_only'})

## Test 4: PURCHASED Relationship (Customer → Product)

The PURCHASED relationship connects customers to products they have bought.

In [8]:
# Test PURCHASED relationship
query = "MATCH (c:customer)-[r:PURCHASED]->(p:product) RETURN c.name AS customer, p.name AS product LIMIT 5"
test_cypher_query(query, "Testing PURCHASED relationship between customers and products")


Testing PURCHASED relationship between customers and products
Query: MATCH (c:customer)-[r:PURCHASED]->(p:product) RETURN c.name AS customer, p.name AS product LIMIT 5
--------------------------------------------------------------------------------
Status Code: 200
Response: {
  "cypher_query": "MATCH (c:customer)-[r:PURCHASED]->(p:product) RETURN c.name AS customer, p.name AS product LIMIT 5",
  "generated_sql": "WITH customer_c AS (\n    SELECT \n      u.name, \n      u.user_id\nFROM customer AS t\n), \nPURCHASED_r AS (\n    SELECT \n      c.from_customer AS from_id, \n      product.to_product AS to_id\nFROM PURCHASED AS t\nWHERE t.to_id IN (SELECT u.user_id FROM customer_c AS t)\n)\nSELECT \n      c.name AS customer, \n      p.name AS product\nFROM product AS p\nINNER JOIN PURCHASED_r AS r ON r.from_id = p.product_id\nINNER JOIN customer_c AS c ON c.user_id = r.to_id\nLIMIT  5",
  "execution_mode": "sql_only"
}
Status Code: 200
Response: {
  "cypher_query": "MATCH (c:customer)-[r:P

(True,
 {'cypher_query': 'MATCH (c:customer)-[r:PURCHASED]->(p:product) RETURN c.name AS customer, p.name AS product LIMIT 5',
  'generated_sql': 'WITH customer_c AS (\n    SELECT \n      u.name, \n      u.user_id\nFROM customer AS t\n), \nPURCHASED_r AS (\n    SELECT \n      c.from_customer AS from_id, \n      product.to_product AS to_id\nFROM PURCHASED AS t\nWHERE t.to_id IN (SELECT u.user_id FROM customer_c AS t)\n)\nSELECT \n      c.name AS customer, \n      p.name AS product\nFROM product AS p\nINNER JOIN PURCHASED_r AS r ON r.from_id = p.product_id\nINNER JOIN customer_c AS c ON c.user_id = r.to_id\nLIMIT  5',
  'execution_mode': 'sql_only'})

## Test 5: Complex Multi-Hop Queries

Testing complex queries that traverse multiple relationships.

In [9]:
# Test multi-hop query: Find posts liked by followers of a user
query = """MATCH (u:user)-[f:FOLLOWS]->(follower:user)-[l:LIKED]->(p:post) 
           RETURN u.name AS user, follower.name AS follower, p.title AS liked_post LIMIT 5"""
test_cypher_query(query, "Testing multi-hop query: user -> follows -> liked posts")

# Test query with relationship properties (if supported)
query2 = "MATCH (u:user)-[r:AUTHORED {published: true}]->(p:post) RETURN u.name, p.title LIMIT 3"
test_cypher_query(query2, "Testing relationship with properties (if supported)")


Testing multi-hop query: user -> follows -> liked posts
Query: MATCH (u:user)-[f:FOLLOWS]->(follower:user)-[l:LIKED]->(p:post) 
           RETURN u.name AS user, follower.name AS follower, p.title AS liked_post LIMIT 5
--------------------------------------------------------------------------------
Status Code: 200
Response: {
  "cypher_query": "MATCH (u:user)-[f:FOLLOWS]->(follower:user)-[l:LIKED]->(p:post) \n           RETURN u.name AS user, follower.name AS follower, p.title AS liked_post LIMIT 5",
  "generated_sql": "WITH user_u AS (\n    SELECT \n      u.name, \n      u.user_id\nFROM user AS t\n), \nFOLLOWS_f AS (\n    SELECT \n      u.from_user AS from_id, \n      u.to_user AS to_id\nFROM FOLLOWS AS t\nWHERE t.to_id IN (SELECT u.user_id FROM user_u AS t)\n), \nuser_follower AS (\n    SELECT \n      u.name, \n      u.user_id\nFROM user AS t\nWHERE u.user_id IN (SELECT t.from_id FROM FOLLOWS_f AS t)\n), \nLIKED_l AS (\n    SELECT \n      u.from_user AS from_id, \n      p.to_post A

(True,
 {'cypher_query': 'MATCH (u:user)-[r:AUTHORED {published: true}]->(p:post) RETURN u.name, p.title LIMIT 3',
  'generated_sql': 'WITH AUTHORED_r AS (\n    SELECT \n      p.title, \n      p.post_id\nFROM post AS t\nWHERE p.post_id IN (SELECT t.to_id FROM AUTHORED_r AS t)\n), \npost_p AS (\n    SELECT \n      u.from_user AS from_id, \n      p.to_post AS to_id\nFROM AUTHORED AS t\nWHERE p.published = true\n)\nSELECT \n      u.name, \n      p.title\nFROM user AS u\nINNER JOIN AUTHORED_r AS r ON r.from_id = u.user_id\nINNER JOIN post_p AS p ON p.post_id = r.to_id\nLIMIT  3',
  'execution_mode': 'sql_only'})

# Variable-Length Path Testing

Testing variable-length relationship patterns using the `*` syntax for multi-hop traversals.

## Test 6: Variable-Length Path (1-3 hops)

Testing variable-length patterns with min and max hop limits.
Syntax: `(a)-[*1..3]->(b)` means 1 to 3 hops between nodes.

In [ ]:
# Test variable-length path: Find users connected through 1-3 FOLLOWS relationships
query = """MATCH (u1:user)-[*1..3]->(u2:user) 
           RETURN u1.name AS start_user, u2.name AS end_user LIMIT 10"""
success, result = test_cypher_query(query, "Testing variable-length path: 1-3 hops")

if success and result:
    sql = result.get('generated_sql', '')
    print("\n📊 SQL Analysis:")
    print("- Contains WITH clause:", "WITH" in sql)
    print("- Contains RECURSIVE:", "RECURSIVE" in sql)
    print("- Contains UNION ALL:", "UNION ALL" in sql)
    print("- Contains hop_count:", "hop_count" in sql)

## Test 7: Fixed-Length Path (*2)

Testing fixed-length patterns that require exactly N hops.
Syntax: `(a)-[*2]->(b)` means exactly 2 hops between nodes.

In [ ]:
# Test fixed-length path: Friend of friend (exactly 2 hops)
query = """MATCH (u1:user)-[*2]->(u2:user) 
           RETURN u1.name AS user, u2.name AS friend_of_friend LIMIT 10"""
success, result = test_cypher_query(query, "Testing fixed-length path: exactly 2 hops")

if success and result:
    sql = result.get('generated_sql', '')
    print("\n📊 Fixed-Length Path Validation:")
    print("- SQL generated:", len(sql) > 0)
    # For fixed-length paths, we expect simpler SQL without recursion
    print("- Uses simple JOINs (expected for *2):", "JOIN" in sql)

## Test 8: Upper-Bounded Path (*..5)

Testing upper-bounded patterns with no minimum (defaults to 1).
Syntax: `(a)-[*..5]->(b)` means 1 to 5 hops between nodes.

In [ ]:
# Test upper-bounded path: Users reachable within 5 hops
query = """MATCH (u1:user)-[*..5]->(u2:user) 
           RETURN u1.name AS start_user, u2.name AS reachable_user LIMIT 10"""
success, result = test_cypher_query(query, "Testing upper-bounded path: up to 5 hops")

if success and result:
    sql = result.get('generated_sql', '')
    print("\n📊 Upper-Bounded Path Analysis:")
    print("- Contains max hop limit (5):", "< 5" in sql or "<= 5" in sql)
    print("- Has recursive structure:", "RECURSIVE" in sql or "UNION" in sql)

## Test 9: Unbounded Path (*)

Testing unbounded patterns (with reasonable default limit).
Syntax: `(a)-[*]->(b)` means any number of hops (typically limited to prevent infinite loops).

In [ ]:
# Test unbounded path: All users reachable through any number of FOLLOWS
query = """MATCH (u1:user)-[*]->(u2:user) 
           RETURN u1.name AS start_user, u2.name AS reachable_user LIMIT 10"""
success, result = test_cypher_query(query, "Testing unbounded path: unlimited hops (with default limit)")

if success and result:
    sql = result.get('generated_sql', '')
    print("\n📊 Unbounded Path Analysis:")
    print("- Has default max limit:", any(str(i) in sql for i in [10, 15, 20, 50]))
    print("- Contains cycle detection:", "has(" in sql.lower() or "path_nodes" in sql)

## Test 10: Cross-Type Variable-Length Paths

Testing variable-length paths across different relationship types.
This tests the complexity of handling heterogeneous path patterns.

In [ ]:
# Test variable-length path with specific relationship type: FOLLOWS
query = """MATCH (u1:user)-[:FOLLOWS*1..3]->(u2:user) 
           RETURN u1.name AS follower, u2.name AS followed LIMIT 10"""
success, result = test_cypher_query(query, "Testing typed variable-length path: FOLLOWS only")

# Test variable-length path with different relationship: AUTHORED
query2 = """MATCH (u:user)-[:AUTHORED*1..2]->(p:post) 
            RETURN u.name AS author, p.title AS post LIMIT 5"""
success2, result2 = test_cypher_query(query2, "Testing variable-length with AUTHORED (may not make sense semantically)")

## Test 11: Edge Cases & Error Handling

Testing edge cases and boundary conditions for variable-length paths.

In [ ]:
# Edge Case 1: Zero hops (should be invalid or equivalent to direct match)
query = """MATCH (u1:user)-[*0]->(u2:user) 
           RETURN u1.name, u2.name LIMIT 5"""
success, result = test_cypher_query(query, "Edge Case: Zero hops (*0)")

# Edge Case 2: Very large hop count (performance test)
query2 = """MATCH (u1:user)-[*1..100]->(u2:user) 
            RETURN u1.name, u2.name LIMIT 5"""
success2, result2 = test_cypher_query(query2, "Edge Case: Large hop count (1..100)")

# Edge Case 3: Inverted range (should be invalid)
query3 = """MATCH (u1:user)-[*5..2]->(u2:user) 
            RETURN u1.name, u2.name LIMIT 5"""
success3, result3 = test_cypher_query(query3, "Edge Case: Inverted range (*5..2) - should fail")

# Edge Case 4: Single minimum hop
query4 = """MATCH (u1:user)-[*1..1]->(u2:user) 
            RETURN u1.name, u2.name LIMIT 5"""
success4, result4 = test_cypher_query(query4, "Edge Case: Single hop range (*1..1) - equivalent to regular pattern")

## Variable-Length Path Test Summary

### Expected SQL Generation Patterns:
For variable-length path queries, the generated SQL should include:
- **WITH RECURSIVE clause**: For iterative path traversal
- **Base case**: Initial single-hop relationships
- **Recursive case**: Extends paths by one hop in each iteration
- **Hop counting**: `hop_count` field tracking path length
- **Cycle detection**: Using `has()` function or path tracking to prevent infinite loops
- **Hop limits**: WHERE clauses enforcing min/max hop constraints
- **UNION ALL**: Combining results from different hop counts

### Test Coverage:
- ✅ Test 6: Variable-length range (1-3 hops)
- ✅ Test 7: Fixed-length paths (*2)
- ✅ Test 8: Upper-bounded paths (*..5)
- ✅ Test 9: Unbounded paths (*)
- ✅ Test 10: Typed variable-length paths
- ✅ Test 11: Edge cases and error handling

### Next Steps:
1. Fix compilation errors in Rust code
2. Start the ClickGraph server
3. Execute all test cells
4. Validate SQL generation for each pattern
5. Document any failures or unexpected behaviors